<a href="https://colab.research.google.com/github/ronenbendavid/IDC_NLP/blob/master/Ronen_Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 1
In this assignment you will be creating tools for learning and testing language models.
The corpora that you will be working with are lists of tweets in 8 different languages that use the Latin script. The data is provided either formatted as CSV or as JSON, for your convenience. The end goal is to write a set of tools that can detect the language of a given tweet.


*As a preparation for this task, place the data files somewhere in your drive so that you can access the files from this notebook. The files are available to download from the Moodle assignment activity*

The relevant files are:


*   en.csv (or the equivalent JSON file)
*   es.csv (or the equivalent JSON file)
*   fr.csv (or the equivalent JSON file)
*   in.csv (or the equivalent JSON file)
*   it.csv (or the equivalent JSON file)
*   nl.csv (or the equivalent JSON file)
*   pt.csv (or the equivalent JSON file)
*   tl.csv (or the equivalent JSON file)





**Part 1**

Write a function *preprocess* that iterates over all the data files and creates a single vocabulary, containing all the tokens in the data. **Our token definition is a single UTF-8 encoded character**. So, the vocabulary list is a simple Python list of all the characters that you see at least once in the data.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
from pathlib import Path
from glob import glob
import csv
import json
import pandas as pd
def preprocess():
  # TODO
 vocabulary = set()
 pathlist = glob('/content/drive/My Drive/IDC/NLP/Assignment1/*.csv')
 for path in pathlist:
    with open(path, encoding='utf-8', newline='') as csv_file:
        csv_reader = pd.read_csv(csv_file, delimiter=',')
        line_count = 0
        for text in csv_reader['tweet_text']:
            list_text = list(text)
            vocabulary.update(list_text)
            line_count += 1
        #print(line_count)
 return sorted(list(vocabulary))

vocabulary = preprocess()
print(len(vocabulary))
print(vocabulary)
  

1859
['\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '^', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}', '~', '\x7f', '\x80', '\x91', '\x92', '\x97', '\x9d', '¡', '£', '¤', '¥', '§', '¨', '©', 'ª', '«', '\xad', '®', '¯', '°', '²', '³', '´', '¶', '·', '¸', 'º', '»', '½', '¾', '¿', 'À', 'Á', 'Â', 'Ã', 'Å', 'Ç', 'È', 'É', 'Ê', 'Ë', 'Ì', 'Í', 'Î', 'Ñ', 'Ò', 'Ó', 'Ô', 'Õ', 'Ö', '×', 'Ù', 'Ú', 'Ü', 'à', 'á', 'â', 'ã', 'ä', 'ç', 'è', 'é', 'ê', 'ë', 'ì', 'í', 'î', 'ï', 'ð', 'ñ', 'ò', 'ó', 'ô', 'õ', 'ö', 'ø', 'ù', 'ú', 'û', 'ü', 'ė', 'Ğ', 'ğ', 'İ', 'ı', 'ń', 'ō', 'Œ', 'œ', 'Ş', 'ş', 'Š', 'Ÿ', 'ƒ', 'ʔ', 'ʕ', 'ʖ', 'ʰ', 'ʳ', '

**Part 2**

Write a function lm that generates a language model from a textual corpus. The function should return a dictionary (representing a model) where the keys are all the relevant n-1 sequences, and the values are dictionaries with the n_th tokens and their corresponding probabilities to occur. For example, for a trigram model (tokens are characters), it should look something like:

{
  "ab":{"c":0.5, "b":0.25, "d":0.25},
  "ca":{"a":0.2, "b":0.7, "d":0.1}
}

which means for example that after the sequence "ab", there is a 0.5 chance that "c" will appear, 0.25 for "b" to appear and 0.25 for "d" to appear.

Note - You should think how to add the add_one smoothing information to the dictionary and implement it.

In [5]:
def find_ngrams(text, length):
    counter = Counter()
    ngram = []
    num_tokens, tokens = split_into_tokens(text.lower())
    for token in tokens:
        for i,_ in enumerate(token):
           if(i>=length - 1):
                post = token[i]
                prior = token[i - (length - 1) : i]
                ngram = ''.join(prior) + post
                counter[ngram] = counter.get(ngram, 0) + 1
    return counter
def token_to_unigram(token):
    token = token.strip().strip(",.!|&-_()[]<>{}/\"'").strip()

    def has_no_chars(token):
        for char in token:
            if char.isalpha():
                return False
        return True

    if len(token) == 1 or token.isdigit() or has_no_chars(token):
        return None
    return token
def split_into_tokens(text):
    tokens = []
    for token in nltk.tokenize.WhitespaceTokenizer().tokenize(text):
        unigram = token_to_unigram(token)
        if unigram:
            tokens.append(unigram)
    return len(tokens), tokens
def lm(n, vocabulary, data_file_path, add_one):
  # n - the n-gram to use (e.g., 1 - unigram, 2 - bigram, etc.)
  # vocabulary - the vocabulary list (which you should use for calculating add_one smoothing)
  # data_file_path - the data_file from which we record probabilities for our model
  # add_one - True/False (use add_one smoothing or not)

  # TODO
  model = {}
  with open(data_file_path, encoding='utf-8', newline='') as csv_file:
    csv_reader = pd.read_csv(csv_file, delimiter=',')
    corpus = []
    for tweet_text in csv_reader['tweet_text']:
            corpus.append(tweet_text)
    data = find_ngrams(" ".join(corpus),n)
    unigram_total = float(sum(data.values()))
    temp_dic = {}

    for ngram in data.items():
        if (add_one == True):
            p = float(ngram[1] + 1)/(unigram_total + len(vocabulary))
        else:
            p = float(ngram[1])/unigram_total
        model.update({ngram[0]:p})
  return model

model = lm(3, vocabulary, '/content/drive/My Drive/IDC/NLP/Assignment1/en.csv.json', True)
print({k: v for i, (k, v) in enumerate(model.items()) if i < 50})
  # n - the n-gram to use (e.g., 1 - unigram, 2 - bigram, etc.)
  # vocabulary - the vocabulary list (which you should use for calculating add_one smoothing)
  # data_file_path - the data_file from which we record probabilities for our model
  # add_one - True/False (use add_one smoothing or not)

  # TODO
  # model = {}

  # corpus_words = sorted(list(set([word for words_list in corpus for word in words_list])))
  # num_corpus_words = len(corpus_words)
  # data = []
  # with open(data_file_path) as csv_file:
  #     csv_reader = pd.read_csv(csv_file, delimiter=',')
  #       for corpus in csv_reader['tweet_text']:
  #         data = sorted(list(set([word for words_list in corpus.split(" ")]
  # #       for word in list(data)
  # #      data =  [[START_TOKEN] + [w.lower() for w in list(data.words(f))] + [END_TOKEN] for f in files]
  # #     data = (" " * n) + data
  # #     for i in range(len(data)-n):
  # #       word, char = data[i:i+n], data[i+n]
  # #       if not word in model:
  # #         model[word] = {}
  # #       model[word][char] = model[word].get(char, 0) + 1
  # # word_freq = Counter(chain(*data))
  # return model

KeyboardInterrupt: ignored

**Part 3**

Write a function *eval* that returns the perplexity of a model (dictionary) running over a given data file.

In [0]:
def eval(n, model, data_file):
  # n - the n-gram that you used to build your model (must be the same number)
  # model - the dictionary (model) to use for calculating perplexity
  # data_file - the tweets file that you wish to claculate a perplexity score for

  # TODO

**Part 4**

Write a function *match* that creates a model for every relevant language, using a specific value of *n* and *add_one*. Then, calculate the perplexity of all possible pairs (e.g., en model applied on the data files en ,es, fr, in, it, nl, pt, tl; es model applied on the data files en, es...). This function should return a pandas DataFrame with columns [en ,es, fr, in, it, nl, pt, tl] and every row should be labeled with one of the languages. Then, the values are the relevant perplexity values.

In [0]:
def match(n, add_one):
  # n - the n-gram to use for creating n-gram models
  # add_one - use add_one smoothing or not

  #TODO

**Part 5**

Run match with *n* values 1-4, once with add_one and once without, and print the 8 tables to this notebook, one after another.

In [0]:
# TODO

# **Good luck!**